<a href="https://colab.research.google.com/github/Hubert26/suicides_IPPAN/blob/main/ml_analize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Liblaries and settings

In [775]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
import os

!pip install -U dtreeviz
import dtreeviz

from google.colab import files

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# avoid "Arial font not found warnings"
import logging
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

np.random.seed(42)

#Function definitions

##plot_confusion_matrix

In [776]:
def plot_confusion_matrix(cm, model):
    # klasyfikacja binarna
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(
        width=500,
        height=500,
        title=model+' Confusion Matrix',
        font_size=16,
        template='plotly_dark'
        )
    fig.show()
    return fig

##plot_roc_curve

**pogrubiony tekst**
 Receiver Operating Characteristic
1. True Positive Rate (TPR) = FP / (FP+TN)
2. False Positive Rate (FPR) = TP / (TP+FN)

In [777]:
def plot_roc_curve(y_test, y_pred, model):
    # Binary classification
    from sklearn.metrics import roc_curve
    fpr, tpr, tresh = roc_curve(y_test, y_pred, pos_label=1)

    fig = go.Figure(
        data=[
            go.Scatter(x=fpr, y=tpr,
                    line_color='red',
                    name='ROC Curve'),
            go.Scatter(x=[0, 1], y=[0, 1],
                    mode='lines',
                    line_dash='dash',
                    line_color='#F012BE')
        ],
        layout=go.Layout(xaxis_title='False Positive Rate',
                         yaxis_title='True Positive Rate',
                         title = model + ' ROC Curve',
                         showlegend=False,
                         template='plotly_dark',
                         width=700))
    fig.show()
    return fig


##model_validation


Precision (precyzja) = TP / (TP+FP): Ile obserwacji przewidzianych jako pozytywne jest w rzeczywistości pozytywne.

Recall (czułość) = TP / (TP+FN): Ile obserwacji z wszystkich pozywtywnych sklasyfikowaliśmy jako pozytywne.

F1-score: ważona średnia harmonicza między precyzją a czułością;

Support (wsparcie): liczba próbek, które należą do każdej z klas;

Accuracy (dokładność) = (TP+TN) /(TP+TN+FP+FN))

FPR (False Positive Rate) = FP / (FP+TN) [type I error]

FNR (False Negative Rate) = FN / (FN+TP) [type II error]

In [778]:
def model_validation(y_test, y_pred, model):
  from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average="weighted")
  recall = recall_score(y_test, y_pred, average="weighted")
  f1 = f1_score(y_test, y_pred, average="weighted")
  roc_score = roc_auc_score(y_test, y_pred)
  cm = confusion_matrix(y_test, y_pred)
  tn, fp, fn, tp = cm.ravel()
  fpr = fp / (fp + tn) # False Positive Rate - Type I error
  fnr = fn / (fn + tp) # False Negative Rate - Type II error

  model_results = pd.DataFrame([{'accuracy': accuracy,
                                 'precision': precision,
                                 'recall': recall,
                                 'f1': f1,
                                 'AUROC': roc_score,
                                 'fpr': fpr,
                                 'fnr': fnr
                                 }],
                                  index=[str(model)]
                               )
  return model_results

##plot_learning_curve

In [779]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

#Data exploration

In [780]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/cleaned_samobojstwa_2013_2022_2.csv', delimiter=',', low_memory=False)

In [781]:
df_raw.head(5)

,ID_samobójcy,Data,Wiek,Płeć,St_cyw,Poz_edu,Inf_praca,Utrzymanie,Sukces,Miejsce,Sposób,Stan,Używanie,Leczenie,Data_rok,Data_miesiąc,L_powodów,Powód_Inne,Powód_Utrata_utrzymania,Powód_Śmierć_bliskiego,Powód_Praca,Powód_Zdrowie,Powód_Bezdomność,Powód_Finanse,Powód_Choroba,Powód_Konflikty_inni,Powód_Mobbing,Powód_Kalectwo,Powód_Choroby_fiz,Powód_Choroby_psych,Powód_Miłość,Powód_HIV/AIDS,Powód_Konflikty_rodzinne,Powód_Ciąża,Powód_Przestępstwa
0,"114771204,00",5.2013,07-12,K,Samotność,Podst,Uczeń,Zależny,0.0,Park,Samookaleczenie_inne,Alko,Nie_zażywał,Nie_leczony,2013,5,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"114815301,00",6.2013,07-12,K,Samotność,Podst_niepełne,Uczeń,Zależny,0.0,Park,Inny,Trzeźwy,Nie_zażywał,Nie_leczony,2013,6,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,"114936744,00",10.2013,07-12,K,Samotność,Podst_niepełne,Uczeń,Zależny,1.0,Mieszkanie,Powieszenie,Trzeźwy,Nie_zażywał,Nie_leczony,2013,10,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"114944870,00",10.2013,07-12,M,Samotność,Podst,Uczeń,Zależny,0.0,Inne,Inny,Trzeźwy,Nie_zażywał,Nie_leczony,2013,10,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,"114948230,00",10.2013,07-12,K,Samotność,Podst_niepełne,Uczeń,Zależny,1.0,Piwnica,Powieszenie,Leki,Nie_zażywał,Nie_leczony,2013,10,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##Filters and file name

['07-12',
 '13-18',
 '19-24',
 '25-29',
 '30-34',
 '35-39',
 '40-44',
 '45-49',
 '50-54',
 '55-59',
 '60-64',
 '65-69',
 '70-74',
 '75-79',
 '80-84',
 '85+']

 sorted(list(set(df_raw['Wiek'])))

['K', 'M']

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

sorted(list(set(df_raw['Data_rok'])))

In [782]:
age_group = sorted(list(set(df_raw['Wiek'])))
gender = ['K', 'M']
year = sorted(list(set(df_raw['Data_rok'])))

In [783]:
age_group_file_title = "age_group_[" + ''.join(filter(str.isdigit, age_group[0][:3])) + "-" + ''.join(filter(str.isdigit, age_group[-1][-3:])) + "]"
year_group_file_title = "year_[" + str(year[0]) + '-' + str(year[-1]) + "]"

In [784]:
file_title = age_group_file_title + '_' + "gender_" + str(gender) + "_" + year_group_file_title
file_title

"age_group_[07-85]_gender_['K', 'M']_year_[2013-2022]"

##Data and NaN exploration

In [785]:
df_data = df_raw[df_raw['Wiek'].isin(age_group) & df_raw['Płeć'].isin(gender) & df_raw['Data_rok'].isin(year)]

In [786]:
df_data.shape

(111908, 35)

In [787]:
df_data['Sukces'].value_counts()

0.0    58238
1.0    53670
Name: Sukces, dtype: int64

In [788]:
##Brakujące dane
total = df_data.isnull().sum().sort_values(ascending=False)
percent = 100*(df_data.isnull().sum()/df_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [789]:
missing_data.head(5)

,Total,Percent
ID_samobójcy,0,0.0
Powód_Mobbing,0,0.0
Powód_Praca,0,0.0
Powód_Zdrowie,0,0.0
Powód_Bezdomność,0,0.0


##Dropping columns

In [790]:
df_data.drop(['ID_samobójcy', 'Data'], inplace=True,	 axis=1, errors='ignore')

In [791]:
df_data['Data_miesiąc'] = df_data['Data_miesiąc'].astype(str)
df_data['Data_rok'] = df_data['Data_rok'].astype(str)

#DTC

##Dummies

In [792]:
data_dummies = pd.get_dummies(df_data)

In [793]:
data_dummies.shape

(111908, 148)

In [794]:
data_dummies = data_dummies.astype(np.uint8)

In [795]:
data_dummies.head()

,Sukces,L_powodów,Powód_Inne,Powód_Utrata_utrzymania,Powód_Śmierć_bliskiego,Powód_Praca,Powód_Zdrowie,Powód_Bezdomność,Powód_Finanse,Powód_Choroba,Powód_Konflikty_inni,Powód_Mobbing,Powód_Kalectwo,Powód_Choroby_fiz,Powód_Choroby_psych,Powód_Miłość,Powód_HIV/AIDS,Powód_Konflikty_rodzinne,Powód_Ciąża,Powód_Przestępstwa,Wiek_07-12,Wiek_13-18,Wiek_19-24,Wiek_25-29,Wiek_30-34,Wiek_35-39,Wiek_40-44,Wiek_45-49,Wiek_50-54,Wiek_55-59,Wiek_60-64,Wiek_65-69,Wiek_70-74,Wiek_75-79,Wiek_80-84,Wiek_85+,Płeć_K,Płeć_M,St_cyw_Małżeństwo,St_cyw_Partnerstwo,St_cyw_Rozwód,St_cyw_Samotność,St_cyw_Separacja,St_cyw_Wdowieństwo,Poz_edu_Gimnazjalne,Poz_edu_Podst,Poz_edu_Podst_niepełne,Poz_edu_Policealne,Poz_edu_Wyższe,Poz_edu_Zawodowe,Poz_edu_Średnie,Inf_praca_Bezrobotny,Inf_praca_Dorywcza,Inf_praca_Fizyczna,Inf_praca_Rolnik,Inf_praca_Stała,Inf_praca_Student,Inf_praca_Uczeń,Inf_praca_Umysłowa,Inf_praca_Własna,Utrzymanie_Bez_dochodu,Utrzymanie_Praca,Utrzymanie_Zależny,Utrzymanie_Świadczenia,Miejsce_Droga,Miejsce_Inne,Miejsce_Izolacja,Miejsce_Lecznica,Miejsce_Mieszkanie,Miejsce_Opieka,Miejsce_Park,Miejsce_Piwnica,Miejsce_Policja,Miejsce_Praca,Miejsce_Szkoła,Miejsce_Tory,Miejsce_Woda,Miejsce_Wojsko,Miejsce_Zabudowania,Sposób_Dopalacze,Sposób_Gaz,Sposób_Inne_leki,Sposób_Inny,Sposób_Odurzające,Sposób_Pod_pojazd,Sposób_Podpalenie,Sposób_Powieszenie,Sposób_Samookaleczenie_inne,Sposób_Samookaleczenie_pow.,Sposób_Trucizna,Sposób_Uduszenie,Sposób_Układ_krwionośny,Sposób_Utonięcie,Sposób_Z_wysokości,Sposób_Zastrzelenie,Sposób_Środki_chemiczne,Sposób_Środki_nasenne,Stan_Alko,Stan_Alko+dopa,Stan_Alko+dopa+odurza,Stan_Alko+leki,Stan_Alko+leki+dopa+odurza,Stan_Alko+leki+odurza,Stan_Alko+narko,Stan_Alko+odurza,Stan_Dopa,Stan_Dopa+odurza,Stan_Leki,Stan_Leki+dopa,Stan_Leki+dopa+odurza,Stan_Leki+odurza,Stan_Narko,Stan_Odurza,Stan_Trzeźwy,Używanie_Alko,Używanie_Alko+dopa,Używanie_Alko+narko,Używanie_Alko+narko+dopa,Używanie_Dopa,Używanie_Narko,Używanie_Narko+dopa,Używanie_Nie_zażywał,Leczenie_Alko,Leczenie_Alko+narko,Leczenie_Narko,Leczenie_Nie_leczony,Data_rok_2013,Data_rok_2014,Data_rok_2015,Data_rok_2016,Data_rok_2017,Data_rok_2018,Data_rok_2019,Data_rok_2020,Data_rok_2021,Data_rok_2022,Data_miesiąc_1,Data_miesiąc_10,Data_miesiąc_11,Data_miesiąc_12,Data_miesiąc_2,Data_miesiąc_3,Data_miesiąc_4,Data_miesiąc_5,Data_miesiąc_6,Data_miesiąc_7,Data_miesiąc_8,Data_miesiąc_9
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


##Train and test split

In [796]:
Y = data_dummies['Sukces']
X = data_dummies.drop('Sukces', axis=1, errors='ignore')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

##Wages

In [797]:
from sklearn.utils.class_weight import compute_class_weight

# Obliczenie wag klas
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Utworzenie słownika wag klas
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# Inicjalizacja modelu Random Forest z ręcznie ustawionymi wagami klas
dtc = DecisionTreeClassifier(class_weight=class_weight_dict, random_state=42)

# Dopasowanie modelu do danych
dtc.fit(X_train, y_train)


DecisionTreeClassifier(class_weight={0: 0.9644791923171633,
                                     1: 1.038237243207422},
                       random_state=42)

##Grid Search

In [798]:
# Definicja siatki parametrów do przeszukiwania
param_grid = {
    'max_depth': [None, 10, 20, 30],  # Maksymalna głębokość drzewa
    'min_samples_split': [2, 5, 10],  # Minimalna liczba próbek do podziału węzła
    'min_samples_leaf': [1, 2, 4]  # Minimalna liczba próbek w liściu
}

# Inicjalizacja przeszukiwania
grid_search = GridSearchCV(dtc, param_grid=param_grid, cv=5)

# Przeszukiwanie przestrzeni parametrów
grid_search.fit(X_train, y_train)

# Najlepsze znalezione parametry
best_params = grid_search.best_params_
print("Najlepsze parametry:", best_params)

# Ostateczna ocena modelu na zbiorze testowym
accuracy = grid_search.score(X_test, y_test)
print("Dokładność na zbiorze testowym:", accuracy)

Najlepsze parametry: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2}
Dokładność na zbiorze testowym: 0.8069877580198374


##Decision Tree Classifier

In [799]:
dtc = DecisionTreeClassifier(
                              class_weight=class_weight_dict,
                              random_state=42,
                              max_depth=best_params['max_depth'],
                              min_samples_split=best_params['min_samples_split'],
                              min_samples_leaf=best_params['min_samples_leaf'])

dtc.fit(X_train, y_train)
dtc_y_pred = dtc.predict(X_test)

##Confusion Matrix

In [800]:
cm = confusion_matrix(y_test, dtc_y_pred)
cm_df = pd.DataFrame(cm, columns=["pred_0", "pred_1"], index=["true_0", "true_1"])
fig_cm = plot_confusion_matrix(cm,'Decision Tree')

##Clasification report

In [801]:
report = classification_report(y_test, dtc_y_pred, output_dict=True)

# Konwersja słownika do DataFrame
report_df = pd.DataFrame(report).transpose()
report_df = report_df.loc[["0", "1"]]

##ROC Curve

In [802]:
fig_roc = plot_roc_curve(y_test, dtc_y_pred, 'Decision Tree')

##Validation

In [803]:
dtc_mv = model_validation(y_test, dtc_y_pred, 'DTC_MV')

In [804]:
class_weights_df = pd.DataFrame([class_weight_dict])
class_weights_df.columns = ['class_0_weight', 'class_1_weight']

In [805]:
train_test_info_df = pd.DataFrame({
    'Samples': [df_data.shape[0]],
    'Columns': [df_data.shape[1]],
    'Dummies': [data_dummies.shape[1]],
    'Sukces_0': [df_data['Sukces'].value_counts().get(0, 0)],
    'Sukces_1': [df_data['Sukces'].value_counts().get(1, 0)],
    'train_samples': [X_train.shape[0]],
    'test_samples': [X_test.shape[0]],
    'y_train_0': y_train.value_counts()[0],
    'y_train_1': y_train.value_counts()[1],
    'y_test_0': y_test.value_counts()[0],
    'y_test_1': y_test.value_counts()[1]
})

In [806]:
params_df = pd.concat([train_test_info_df, class_weights_df, pd.DataFrame([best_params])], axis=1).T
params_df

,0
Samples,111908.000000
Columns,33.000000
Dummies,148.000000
Sukces_0,58238.000000
Sukces_1,53670.000000
train_samples,78335.000000
test_samples,33573.000000
y_train_0,40610.000000
y_train_1,37725.000000
y_test_0,17628.000000


In [807]:
importances = dtc.feature_importances_

# Sortowanie cech według ważności
sorted_indices = importances.argsort()[::-1]
data = {'Feature': [X.columns[index] for index in sorted_indices],
        'Importance': [importances[index] for index in sorted_indices]}

importances_df = pd.DataFrame(data)

In [808]:
importances_df

,Feature,Importance
0,Sposób_Powieszenie,0.689373
1,Sposób_Z_wysokości,0.042832
2,Powód_Konflikty_rodzinne,0.039606
3,Sposób_Zastrzelenie,0.031398
4,Powód_Miłość,0.026707
5,Utrzymanie_Świadczenia,0.025922
6,Miejsce_Tory,0.024953
7,Sposób_Utonięcie,0.010485
8,Miejsce_Mieszkanie,0.006901
9,Sposób_Uduszenie,0.006876


## Decission Tree

###dtreeviz

In [809]:
viz_model = dtreeviz.model(dtc,
                           X_train = X_train,
                           y_train = y_train,
                           feature_names=list(X_train.columns),
                           target_name='suicides')
v = viz_model.view(orientation='LR', leaftype='barh')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names



####Leaf distribution

###graphviz

In [810]:
# # Eksportowanie drzewa do pliku DOT
# export_graphviz(dtc, out_file='tree.dot', feature_names=X_train.columns, filled=True, rounded=True)

# # Konwersja pliku DOT na obraz
# with open('tree.dot') as f:
#     dot_graph = f.read()

##Saveing data

In [811]:
!ls

'DTC_MV_age_group_[85-85]_gender_['\''K'\'', '\''M'\'']_year_[2013-2022].xlsx'
 sample_data
'TREE_age_group_[85-85]_gender_['\''K'\'', '\''M'\'']_year_[2013-2022]'
'TREE_age_group_[85-85]_gender_['\''K'\'', '\''M'\'']_year_[2013-2022].svg'


In [812]:
for file in os.listdir():
    if os.path.isfile(file):
        os.remove(file)

####Saveing params

In [813]:
file_name = "DTC_MV_" + file_title + ".xlsx"

In [814]:
with pd.ExcelWriter(file_name, engine='openpyxl') as writer:

    params_df.to_excel(writer, sheet_name='Model params', index=True)

    # Zapisz DataFrame importances_df na arkuszu 'Importances'
    importances_df.to_excel(writer, sheet_name='Importances', index=False)

    # Zapisz DataFrame report_df na nowym arkuszu 'Classification report'
    report_df.to_excel(writer, sheet_name='Classification report', index=True)

    # Zapisz DataFrame cm_df na nowym arkuszu 'Confusion Matrix'
    cm_df.to_excel(writer, sheet_name='Confusion Matrix', index=True)

# Pobierz plik Excel
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

####Saveing ROC

In [815]:
# # Zapisywanie figury jako pliku HTML
# file_name = "DTC_ROC_CURVE_" + file_title + ".html"
# fig_roc.write_html(file_name)

# # Pobieranie pliku HTML
# files.download(file_name)

####Saveing tree dtreeviz

In [816]:
file_name = "TREE_" + file_title + ".svg"
v.save(file_name)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [817]:
# viz_model.ctree_leaf_distributions()
# plt.tight_layout()
# plt.show()

####Saveing tree graphviz

In [818]:
# file_name = "TREE_" + file_title
# graphviz.Source(dot_graph)
# graphviz.Source(dot_graph).render(file_name, format="svg")
# files.download(file_name + '.svg')